# Notebook 02: AgentCore Runtime Setup

## Learning Objectives
- Configure AgentCore Runtime for travel agent
- Create basic conversational agent using Strands
- Implement multi-turn dialogue patterns
- Test conversation flow and state management

## Prerequisites
- Completed Notebook 01 (Foundation)
- AWS environment validated
- API keys configured

## Step 1: Connect to your AWS environment

In [1]:
import os

os.environ['AWS_REGION'] = 'us-east-1'

# APPROACH A: Use credentials
# os.environ['AWS_ACCESS_KEY_ID'] = 'your_access_key'
# os.environ['AWS_SECRET_ACCESS_KEY'] = 'your_secret_key'
# os.environ['AWS_SESSION_TOKEN'] = "your_session_token"

# APPROACH B: Use AWS SSO profile
#os.environ['AWS_PROFILE'] = 'your_profile'
# Remove any existing credential env vars to force profile usage
#for key in ['AWS_ACCESS_KEY_ID', 'AWS_SECRET_ACCESS_KEY', 'AWS_SESSION_TOKEN']:
#    os.environ.pop(key, None)

os.environ['AWS_REGION'] = 'us-east-1'

print("✅ AWS Profile set. Please restart kernel and run all cells.")

✅ AWS Profile set. Please restart kernel and run all cells.


In [2]:
import os
from dotenv import load_dotenv
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from strands import Agent, tool
from strands.models import BedrockModel

# Load environment variables
load_dotenv()

print("✅ AgentCore Runtime imports successful")

✅ AgentCore Runtime imports successful


## Step 2: Create Basic Travel Agent Tools

In [3]:
# Define basic travel tools for our agent
@tool
def get_travel_preferences():
    """Get user's travel preferences from memory"""
    # Mock implementation - will be enhanced in Memory notebook
    return {
        "hotel_type": "mid-range",
        "food_preference": "vegetarian",
        "budget_range": "moderate"
    }

@tool
def calculate_budget(total_budget: int, days: int):
    """Calculate daily budget allocation for travel"""
    # Basic budget breakdown
    daily_budget = total_budget / days
    allocation = {
        "flights": total_budget * 0.24,  # 24%
        "hotels": total_budget * 0.36,   # 36%
        "food": total_budget * 0.20,     # 20%
        "activities": total_budget * 0.16, # 16%
        "buffer": total_budget * 0.04     # 4%
    }
    return {
        "daily_budget": daily_budget,
        "allocation": allocation
    }

@tool
def get_destination_info(destination: str):
    """Get basic information about a travel destination"""
    # Mock implementation - will be enhanced with real APIs
    destinations = {
        "rome": {
            "country": "Italy",
            "currency": "EUR",
            "language": "Italian",
            "attractions": ["Colosseum", "Vatican", "Trevi Fountain"]
        },
        "florence": {
            "country": "Italy",
            "currency": "EUR",
            "language": "Italian",
            "attractions": ["Uffizi Gallery", "Ponte Vecchio", "Duomo"]
        },
        "venice": {
            "country": "Italy",
            "currency": "EUR",
            "language": "Italian",
            "attractions": ["St. Mark's Square", "Grand Canal", "Doge's Palace"]
        }
    }
    return destinations.get(destination.lower(), {"error": "Destination not found"})

print("✅ Travel agent tools defined")

✅ Travel agent tools defined


## Step 3: Create Travel Agent with Bedrock Model

In [6]:
# Initialize Bedrock model for the agent
model_id = "amazon.nova-pro-v1:0"
model = BedrockModel(model_id=model_id)

# Create the travel agent with system prompt
system_prompt = """
You are an AI Travel Companion specializing in planning trips to Italy. 
Your expertise includes:
- Flight and hotel recommendations
- Budget optimization and allocation
- Destination information and attractions
- Personalized recommendations based on user preferences

Always ask clarifying questions to better understand the user's needs.
Be helpful, friendly, and provide detailed explanations for your recommendations.
Remember user preferences and reference them in future interactions.
"""

travel_agent = Agent(
    model=model,
    tools=[get_travel_preferences, calculate_budget, get_destination_info],
    system_prompt=system_prompt
)

print("✅ Travel agent created with Bedrock model")

✅ Travel agent created with Bedrock model


## Step 4: Test Local Agent Functionality

In [7]:
# Test the agent locally before deploying to runtime
def test_travel_agent(user_input):
    """Test function for local agent invocation"""
    print(f"User: {user_input}")
    response = travel_agent(user_input)
    agent_response = response.message['content'][0]['text']
    print(f"Agent: {agent_response}")
    return agent_response

# Test basic functionality
print("🧪 Testing Travel Agent Locally")
print("=" * 50)

test_travel_agent("Hi, I want to plan a trip to Italy")

🧪 Testing Travel Agent Locally
User: Hi, I want to plan a trip to Italy
<thinking> 
To plan a trip to Italy, I need to gather some information about the user's preferences, budget, and the duration of the trip. This will help me provide personalized recommendations. 
First, I will use the `get_travel_preferences` tool to check if there are any previously stored preferences for this user.
</thinking>

Tool #1: get_travel_preferences
<thinking> 
The user has previously indicated a preference for mid-range hotels, vegetarian food, and a moderate budget. 
Next, I need to ask the user about the duration of their trip and their specific interests or must-see destinations in Italy.
</thinking>
Hi! It looks like you prefer mid-range hotels, vegetarian food, and have a moderate budget. To help you plan your trip to Italy, could you please provide me with the following information:
1. How many days will you be traveling?
2. Are there any specific cities or regions in Italy you want to visit?
3. 

"<thinking> \nThe user has previously indicated a preference for mid-range hotels, vegetarian food, and a moderate budget. \nNext, I need to ask the user about the duration of their trip and their specific interests or must-see destinations in Italy.\n</thinking>\nHi! It looks like you prefer mid-range hotels, vegetarian food, and have a moderate budget. To help you plan your trip to Italy, could you please provide me with the following information:\n1. How many days will you be traveling?\n2. Are there any specific cities or regions in Italy you want to visit?\n3. Do you have any particular interests or activities you'd like to include in your itinerary (e.g., history, art, cuisine, nature)?"

In [8]:
# Test budget calculation
test_travel_agent("I have a budget of $5000 for a 10-day trip. How should I allocate it?")

User: I have a budget of $5000 for a 10-day trip. How should I allocate it?
<thinking> 
The user has provided their total budget and the duration of their trip. I can use the `calculate_budget` tool to determine a daily budget allocation.
</thinking> 
Tool #2: calculate_budget
<thinking> 
The budget has been allocated as follows:
- Flights: $1200
- Hotels: $1800
- Food: $1000
- Activities: $800
- Buffer: $200
This leaves a daily budget of $500.
</thinking>
Great! Based on your $5000 budget for a 10-day trip, here's how you can allocate your funds:
- **Flights**: Approximately $1200
- **Hotels**: Approximately $1800 (around $180 per night for mid-range hotels)
- **Food**: Approximately $1000 (around $100 per day for vegetarian meals)
- **Activities**: Approximately $800 (around $80 per day for attractions and experiences)
- **Buffer**: $200 for unexpected expenses

This allocation should help you manage your budget effectively. Now, let's talk about your destinations and activities. You

"<thinking> \nThe budget has been allocated as follows:\n- Flights: $1200\n- Hotels: $1800\n- Food: $1000\n- Activities: $800\n- Buffer: $200\nThis leaves a daily budget of $500.\n</thinking>\nGreat! Based on your $5000 budget for a 10-day trip, here's how you can allocate your funds:\n- **Flights**: Approximately $1200\n- **Hotels**: Approximately $1800 (around $180 per night for mid-range hotels)\n- **Food**: Approximately $1000 (around $100 per day for vegetarian meals)\n- **Activities**: Approximately $800 (around $80 per day for attractions and experiences)\n- **Buffer**: $200 for unexpected expenses\n\nThis allocation should help you manage your budget effectively. Now, let's talk about your destinations and activities. You mentioned you want to visit specific cities or regions—could you please share which ones? Additionally, any particular interests or activities you'd like to include in your itinerary?"

In [9]:
# Test destination information
test_travel_agent("Tell me about Rome and what I should see there")

User: Tell me about Rome and what I should see there
<thinking> 
The user wants to know about Rome and what to see there. I will use the `get_destination_info` tool to gather information about Rome.
</thinking> 
Tool #3: get_destination_info
<thinking> 
I have gathered information about Rome, including its attractions. I will now provide the user with detailed information about Rome and recommend must-see attractions.
</thinking>
Rome is a fascinating city rich in history, culture, and cuisine. Here are some must-see attractions:
1. **Colosseum**: An iconic symbol of Rome, this ancient amphitheater hosted gladiatorial contests and public spectacles.
2. **Vatican**: Home to St. Peter's Basilica and the Sistine Chapel, the Vatican is a treasure trove of art and history.
3. **Trevi Fountain**: One of the most famous fountains in the world, it's a beautiful Baroque masterpiece.

Given your preference for vegetarian food, Rome offers a variety of vegetarian restaurants. For mid-range hotels

"<thinking> \nI have gathered information about Rome, including its attractions. I will now provide the user with detailed information about Rome and recommend must-see attractions.\n</thinking>\nRome is a fascinating city rich in history, culture, and cuisine. Here are some must-see attractions:\n1. **Colosseum**: An iconic symbol of Rome, this ancient amphitheater hosted gladiatorial contests and public spectacles.\n2. **Vatican**: Home to St. Peter's Basilica and the Sistine Chapel, the Vatican is a treasure trove of art and history.\n3. **Trevi Fountain**: One of the most famous fountains in the world, it's a beautiful Baroque masterpiece.\n\nGiven your preference for vegetarian food, Rome offers a variety of vegetarian restaurants. For mid-range hotels, you might consider staying in the Centro Storico area, which is close to many attractions.\n\nWould you like more detailed information about any of these attractions, or should I provide recommendations for other cities in Italy?"

## Step 5: Prepare Agent for AgentCore Runtime

In [10]:
%%writefile ../backend/runtime/simple_agent/travel_agent.py
from strands import Agent, tool
from strands.models import BedrockModel
from bedrock_agentcore.runtime import BedrockAgentCoreApp
import json

# Initialize AgentCore Runtime App
app = BedrockAgentCoreApp()

# Define travel tools
@tool
def get_travel_preferences():
    """Get user's travel preferences from memory"""
    return {
        "hotel_type": "mid-range",
        "food_preference": "vegetarian",
        "budget_range": "moderate"
    }

@tool
def calculate_budget(total_budget: int, days: int):
    """Calculate daily budget allocation for travel"""
    daily_budget = total_budget / days
    allocation = {
        "flights": total_budget * 0.24,
        "hotels": total_budget * 0.36,
        "food": total_budget * 0.20,
        "activities": total_budget * 0.16,
        "buffer": total_budget * 0.04
    }
    return {
        "daily_budget": daily_budget,
        "allocation": allocation
    }

@tool
def get_destination_info(destination: str):
    """Get basic information about a travel destination"""
    destinations = {
        "rome": {
            "country": "Italy",
            "currency": "EUR",
            "language": "Italian",
            "attractions": ["Colosseum", "Vatican", "Trevi Fountain"]
        },
        "florence": {
            "country": "Italy",
            "currency": "EUR",
            "language": "Italian",
            "attractions": ["Uffizi Gallery", "Ponte Vecchio", "Duomo"]
        },
        "venice": {
            "country": "Italy",
            "currency": "EUR",
            "language": "Italian",
            "attractions": ["St. Mark's Square", "Grand Canal", "Doge's Palace"]
        }
    }
    return destinations.get(destination.lower(), {"error": "Destination not found"})

# Initialize model and agent
model_id = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"
model = BedrockModel(model_id=model_id)

system_prompt = """
You are an AI Travel Companion specializing in planning trips to Italy. 
Your expertise includes:
- Flight and hotel recommendations
- Budget optimization and allocation
- Destination information and attractions
- Personalized recommendations based on user preferences

Always ask clarifying questions to better understand the user's needs.
Be helpful, friendly, and provide detailed explanations for your recommendations.
Remember user preferences and reference them in future interactions.
"""

travel_agent = Agent(
    model=model,
    tools=[get_travel_preferences, calculate_budget, get_destination_info],
    system_prompt=system_prompt
)

@app.entrypoint
def invoke_travel_agent(payload):
    """AgentCore Runtime entrypoint for travel agent"""
    user_input = payload.get("prompt", "")
    print(f"User input: {user_input}")
    
    response = travel_agent(user_input)
    agent_response = response.message['content'][0]['text']
    
    return agent_response

if __name__ == "__main__":
    app.run()

Overwriting ../backend/runtime/simple_agent/travel_agent.py


In [11]:
%%writefile ../backend/runtime/simple_agent/requirements.txt
# Core Amazon Bedrock AgentCore dependencies
bedrock-agentcore>=1.0.5
bedrock-agentcore-starter-toolkit>=0.1.27

# AWS SDK and utilities
boto3>=1.40.62
python-dotenv>=1.2.1

# Agent framework
strands-agents>=1.14.0

Overwriting ../backend/runtime/simple_agent/requirements.txt


## Step 6: Deploy to AgentCore Runtime

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
import os
from pathlib import Path

# Initialize runtime deployment
boto_session = Session()
region = boto_session.region_name or "us-east-1"

# Change to project root for AgentCore configuration
original_dir = os.getcwd()
os.chdir('../backend/runtime/simple_agent')
project_root = os.getcwd()

agentcore_runtime = Runtime()
agent_name = "travel_companion_basic"

print(f"🚀 Configuring AgentCore Runtime deployment...")
print(f"Agent Name: {agent_name}")
print(f"Region: {region}")
print(f"Project Root: {project_root}")

# Configure with paths relative to project root
configure_response = agentcore_runtime.configure(
    entrypoint="travel_agent.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name=agent_name,
    container_runtime=None  # Force CodeBuild for cross-platform build
)

print("✅ Runtime configuration complete")
print("💡 Using CodeBuild for cross-platform ARM64 deployment")
configure_response

Entrypoint parsed: file=C:\Code\Work\agentcore-demo\capstone_project\backend\runtime\simple_agent\travel_agent.py, bedrock_agentcore_name=travel_agent
Memory disabled - agent will be stateless
Configuring BedrockAgentCore agent: travel_companion_basic


🚀 Configuring AgentCore Runtime deployment...
Agent Name: travel_companion_basic
Region: us-east-1
Project Root: c:\Code\Work\agentcore-demo\capstone_project\backend\runtime\simple_agent


Memory disabled
Network mode: PUBLIC


⚠️ Platform mismatch: Current system is 'linux/amd64' but Bedrock AgentCore requires 'linux/arm64', so local builds
won't work.
Please use default launch command which will do a remote cross-platform build using code build.For deployment other
options and workarounds, see: 
https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/getting-started-custom.html

Generated Dockerfile: Dockerfile
Generated .dockerignore: c:\Code\Work\agentcore-demo\capstone_project\backend\runtime\simple_agent\.dockerignore
Keeping 'travel_companion_basic' as default agent
Bedrock AgentCore configured: c:\Code\Work\agentcore-demo\capstone_project\backend\runtime\simple_agent\.bedrock_agentcore.yaml


✅ Runtime configuration complete
💡 Using CodeBuild for cross-platform ARM64 deployment


ConfigureResult(config_path=WindowsPath('c:/Code/Work/agentcore-demo/capstone_project/backend/runtime/simple_agent/.bedrock_agentcore.yaml'), dockerfile_path=WindowsPath('c:/Code/Work/agentcore-demo/capstone_project/backend/runtime/simple_agent/Dockerfile'), dockerignore_path=WindowsPath('c:/Code/Work/agentcore-demo/capstone_project/backend/runtime/simple_agent/.dockerignore'), runtime='Docker', runtime_type=None, region='us-east-1', account_id='711015674645', execution_role=None, ecr_repository=None, auto_create_ecr=True, s3_path=None, auto_create_s3=False, memory_id=None, network_mode='PUBLIC', network_subnets=None, network_security_groups=None, network_vpc_id=None)

In [7]:
# Handle execution role manually due to permissions boundary restrictions
import boto3

def find_or_create_execution_role():
    """Find existing suitable role or provide instructions to create one manually"""
    iam_client = boto3.client('iam')
    
    # Common role names that might exist in corporate environments
    potential_roles = [
        f"AmazonBedrockAgentCoreSDKRuntime-{region}-{agent_name}",
        "BedrockAgentExecutionRole",
        "LambdaExecutionRole",
        "AmazonBedrockExecutionRole"
    ]
    
    print("🔍 Searching for existing execution roles...")
    
    try:
        # List roles and find suitable ones
        roles_response = iam_client.list_roles(MaxItems=100)
        existing_roles = [role['RoleName'] for role in roles_response['Roles']]
        
        # Look for Bedrock or Lambda execution roles
        suitable_roles = [role for role in existing_roles 
                         if any(keyword in role.lower() for keyword in 
                               ['bedrock', 'lambda', 'execution', 'agentcore'])]
        
        if suitable_roles:
            print(f"✅ Found potential execution roles:")
            for role in suitable_roles[:5]:  # Show first 5
                print(f"   • {role}")
            print(f"\n💡 You can use one of these roles by setting:")
            print(f"   execution_role_arn='arn:aws:iam::{boto3.client('sts').get_caller_identity()['Account']}:role/ROLE_NAME'")
        else:
            print("❌ No suitable execution roles found")
            
    except Exception as e:
        print(f"⚠️ Could not list roles: {e}")
    
    print(f"\n🛠️ Manual Role Creation Required:")
    print(f"Ask your AWS administrator to create a role with:")
    print(f"   • Role Name: AmazonBedrockAgentCoreSDKRuntime-{region}-346339bc6e")
    print(f"   • Trust Policy: Bedrock AgentCore service")
    print(f"   • Policies: AmazonBedrockAgentPermissions, CloudWatchLogsFullAccess")
    
    return None

find_or_create_execution_role()

🔍 Searching for existing execution roles...
✅ Found potential execution roles:
   • AmazonSageMakerDomainExecutionServiceRole
   • AWS-SystemsManager-AutomationExecutionRole
   • Bedrock-poc-access
   • bedrock-runtime-role20250716104658524900000001
   • hp-coreservices-lambda-role-FK8RNR

💡 You can use one of these roles by setting:
   execution_role_arn='arn:aws:iam::711015674645:role/ROLE_NAME'

🛠️ Manual Role Creation Required:
Ask your AWS administrator to create a role with:
   • Role Name: AmazonBedrockAgentCoreSDKRuntime-us-east-1-346339bc6e
   • Trust Policy: Bedrock AgentCore service
   • Policies: AmazonBedrockAgentPermissions, CloudWatchLogsFullAccess


### 💡 Corporate Environment Notes

If you encounter **permissions boundary** restrictions when deploying in corporate AWS environments:

- **Issue**: Corporate security policies may block automatic IAM role creation
- **Solution**: Ask your AWS administrator to pre-create required roles:
  - Runtime Execution Role: `AmazonBedrockAgentCoreSDKRuntime-[region]-[hash]`
  - CodeBuild Execution Role: `AmazonBedrockAgentCoreSDKCodeBuild-[region]-[hash]`

**Alternative Options:**
- Use `agentcore_runtime.launch(local=True)` for local development
- Deploy to AWS Lambda instead (covered in later notebooks)
- Use a personal/development AWS account without restrictions

The agent functionality remains the same - this only affects deployment method.

In [ ]:
# Alternative: Local Development Mode (if AWS deployment fails)
print("🏠 Testing agent locally (no AWS roles required)")

try:
    print("🧪 Starting local AgentCore mode...")
    local_result = agentcore_runtime.launch(local=True)
    print("✅ Local agent launched successfully!")
    
    # Test the local agent
    test_payload = {"prompt": "Hi, I want to plan a trip to Italy with $3000 budget"}
    local_response = agentcore_runtime.invoke(test_payload)
    print(f"User: {test_payload['prompt']}")
    print(f"Agent: {local_response['response'][0][:200]}...")
    
    print("\n✅ Local testing successful!")
    
except Exception as e:
    print(f"❌ Local testing failed: {e}")
    print("💡 Try running Step 4 cells again to test agent functionality")

🏠 Launching Bedrock AgentCore (local mode)...
   • Build and run container locally
   • Requires Docker/Finch/Podman to be installed
   • Perfect for development and testing
Memory disabled - skipping memory creation
Memory disabled - skipping memory creation
Launching Bedrock AgentCore agent 'travel_companion_basic' locally


🏠 Alternative: Testing agent locally (no AWS roles needed)
This bypasses permissions boundary restrictions
🧪 Starting local AgentCore development mode...


Using build directory: c:\Code\Work\agentcore-demo\capstone_project\backend\runtime\simple_agent


In [ ]:
# Launch the agent to AgentCore Runtime
print("🚀 Launching agent to AgentCore Runtime...")
print("This may take 5-10 minutes...")

try:
    launch_result = agentcore_runtime.launch()
    print("✅ Launch initiated")
    print(f"Agent ARN: {launch_result.agent_arn}")
    print(f"ECR URI: {launch_result.ecr_uri}")
    
except Exception as e:
    print("❌ Launch failed!")
    print(f"Error: {e}")
    
    if "role" in str(e).lower() or "permissions" in str(e).lower():
        print("\n💡 This appears to be a permissions issue (common in corporate environments)")
        print("📋 Try the local development mode cell above as an alternative")
        print("📋 Or ask your AWS admin to pre-create the required execution roles")
    else:
        print(f"\n🔍 Unexpected error - check AWS console for details")
        print("📋 Try the local development mode cell above to test agent functionality")

🚀 Launching Bedrock AgentCore (cloud mode - RECOMMENDED)...
   • Deploy Python code directly to runtime
   • No Docker required (DEFAULT behavior)
   • Production-ready deployment

💡 Deployment options:
   • runtime.launch()                → Cloud (current)
   • runtime.launch(local=True)      → Local development
Memory disabled - skipping memory creation
Starting CodeBuild ARM64 deployment for agent 'travel_companion_basic' to account 711015674645 (us-east-1)
Setting up AWS resources (ECR repository, execution roles)...
Getting or creating ECR repository for agent: travel_companion_basic


🚀 Launching agent to AgentCore Runtime...
⚠️ Note: This requires an execution role to be configured
This may take 5-10 minutes...


ECR repository available: 711015674645.dkr.ecr.us-east-1.amazonaws.com/bedrock-agentcore-travel_companion_basic
Using execution role from config: arn:aws:iam::711015674645:role/AmazonBedrockAgentCoreSDKRuntime-us-east-1-346339bc6e
Preparing CodeBuild project and uploading source...


✅ Reusing existing ECR repository: 711015674645.dkr.ecr.us-east-1.amazonaws.com/bedrock-agentcore-travel_companion_basic


Getting or creating CodeBuild execution role for agent: travel_companion_basic
Role name: AmazonBedrockAgentCoreSDKCodeBuild-us-east-1-346339bc6e
CodeBuild role doesn't exist, creating new role: AmazonBedrockAgentCoreSDKCodeBuild-us-east-1-346339bc6e
Creating IAM role: AmazonBedrockAgentCoreSDKCodeBuild-us-east-1-346339bc6e
Error creating role: An error occurred (AccessDenied) when calling the CreateRole operation: User: arn:aws:sts::711015674645:assumed-role/DEVELOPER/shyam.menon@hp.com is not authorized to perform: iam:CreateRole on resource: arn:aws:iam::711015674645:role/AmazonBedrockAgentCoreSDKCodeBuild-us-east-1-346339bc6e with an explicit deny in a permissions boundary: arn:aws:iam::711015674645:policy/DIMEIAMCustomUserPermissionBoundary
Access denied. Ensure your AWS credentials have sufficient IAM permissions to create roles and policies.
Launch failed after creating the following resources: ['ECR Repository: 711015674645.dkr.ecr.us-east-1.amazonaws.com/bedrock-agentcore-trav

❌ Launch failed!
Error: Launch failed. Resources created: ['ECR Repository: 711015674645.dkr.ecr.us-east-1.amazonaws.com/bedrock-agentcore-travel_companion_basic', 'Runtime Execution Role: arn:aws:iam::711015674645:role/AmazonBedrockAgentCoreSDKRuntime-us-east-1-346339bc6e']

💡 This error is likely due to missing execution role configuration
📋 Please follow these steps:
1. Run the 'find_or_create_execution_role()' cell above
2. Get a suitable role ARN from your AWS admin or existing roles
3. Update the configuration with the execution_role_arn parameter
4. Re-run this cell

🔧 Example configuration with execution role:

configure_response = agentcore_runtime.configure(
    entrypoint="travel_agent.py",
    auto_create_execution_role=False,
    execution_role_arn="arn:aws:iam::711015674645:role/YOUR_ROLE_NAME",
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name=agent_name,
    container_runtime=None
)
        


In [ ]:
# Check deployment status
import time

print("⏳ Checking deployment status...")
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']

while status not in end_status:
    print(f"Status: {status}")
    time.sleep(30)  # Check every 30 seconds
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']

print(f"\n🎉 Final Status: {status}")

if status == 'READY':
    print("✅ Agent successfully deployed to AgentCore Runtime!")
else:
    print("❌ Deployment failed. Check AWS console for details.")

# Return to original directory
os.chdir(original_dir)

## Step 7: Test Deployed Agent

In [ ]:
# Test the deployed agent
if status == 'READY':
    print("🧪 Testing deployed Travel Agent")
    print("=" * 50)
    
    # Test basic interaction
    test_payload = {"prompt": "Hi, I want to plan a 10-day trip to Italy with a $5000 budget"}
    invoke_response = agentcore_runtime.invoke(test_payload)
    
    print(f"User: {test_payload['prompt']}")
    print(f"Agent: {invoke_response['response'][0]}")
else:
    print("⚠️ Cannot test - deployment not ready")

In [ ]:
# Test budget calculation functionality
if status == 'READY':
    test_payload = {"prompt": "Can you help me allocate my $5000 budget for 10 days?"}
    invoke_response = agentcore_runtime.invoke(test_payload)
    
    print(f"\nUser: {test_payload['prompt']}")
    print(f"Agent: {invoke_response['response'][0]}")

## Step 8: Multi-turn Conversation Test

In [ ]:
# Test multi-turn conversation
if status == 'READY':
    print("🗣️ Testing Multi-turn Conversation")
    print("=" * 50)
    
    conversation = [
        "I want to visit Italy",
        "I prefer mid-range hotels and vegetarian food",
        "Tell me about Rome's attractions",
        "What about Florence?"
    ]
    
    for i, message in enumerate(conversation, 1):
        print(f"\n--- Turn {i} ---")
        test_payload = {"prompt": message}
        invoke_response = agentcore_runtime.invoke(test_payload)
        
        print(f"User: {message}")
        print(f"Agent: {invoke_response['response'][0][:200]}...")  # Truncate for readability

## Step 9: Save Runtime Information

In [ ]:
# Save runtime information for use in subsequent notebooks
import json

# Save runtime information if deployment was successful
try:
    # Check if we have a successful deployment
    if 'launch_result' in locals() and hasattr(launch_result, 'agent_arn'):
        runtime_info = {
            "agent_name": agent_name,
            "agent_arn": launch_result.agent_arn,
            "agent_id": launch_result.agent_id,
            "ecr_uri": launch_result.ecr_uri,
            "region": region,
            "status": "DEPLOYED"
        }
        print("📝 Saving deployment information...")
    else:
        # Fallback for local testing or failed deployment
        runtime_info = {
            "agent_name": agent_name,
            "region": region,
            "status": "LOCAL_TESTING_ONLY",
            "agent_code_location": "../backend/runtime/simple_agent/travel_agent.py",
            "notes": "Agent functionality verified locally. Use local mode or resolve deployment issues."
        }
        print("📝 Saving local testing information...")
    
    # Ensure environments directory exists
    import os
    os.makedirs('environments', exist_ok=True)
    
    # Save to file for next notebooks
    with open('environments/runtime_info.json', 'w') as f:
        json.dump(runtime_info, f, indent=2)
    
    print("💾 Runtime information saved to environments/runtime_info.json")
    print("\n📋 Summary:")
    for key, value in runtime_info.items():
        print(f"  {key}: {value}")
        
except Exception as e:
    print(f"⚠️ Could not save runtime info: {e}")
    print("💡 This won't affect the next notebook - information can be reconfigured")

## Next Steps

✅ **Completed in this notebook:**
- AgentCore Runtime configuration and setup
- Basic travel agent with Strands and Bedrock
- Multi-turn conversation capabilities
- Local agent testing and validation
- Runtime deployment (AWS) or local development setup

🎯 **Key Learning Outcomes:**
- Created a functional AI travel agent using Strands framework
- Implemented custom tools for budget calculation and destination info
- Tested conversational flow with multiple interaction patterns
- Learned AgentCore deployment process and alternatives

➡️ **Next: Notebook 03 - Gateway Integration**
- Integrate external APIs (flights, hotels, weather, currency)
- Create OpenAPI 3.0 specifications
- Set up MCP Gateway with OAuth
- Test real API integrations with live data

💡 **Note:** Your agent is fully functional whether deployed to AWS or running locally. The next notebook will enhance it with real-world API integrations!